# Setup

In [7]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
%autoreload

import my_functions as mf
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 7)
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['figure.figsize'] = (10, 5)
plt.style.use("seaborn-v0_8-darkgrid")
import seaborn as sns
sns.set(rc={"figure.dpi":200, 'savefig.dpi':200 , 'figure.figsize': (10, 5)})
sns.set_context('notebook')
sns.set_style("ticks")
sns.set_style('darkgrid')

from pycaret.regression import *

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML

In [9]:
Y_train_a, X_train_a, X_test_a, X_a, train_a = mf.full_clean(
    pd.read_parquet('data/A/train_targets.parquet'),
    pd.concat([pd.read_parquet('data/A/X_train_observed.parquet'), pd.read_parquet('data/A/X_train_estimated.parquet').drop("date_calc", axis=1)]),
    pd.read_parquet('data/A/X_test_estimated.parquet').drop("date_calc", axis=1),
    "A"
)

Y_train_b, X_train_b, X_test_b, X_b, train_b = mf.full_clean(
    pd.read_parquet('data/B/train_targets.parquet'),
    pd.concat([pd.read_parquet('data/B/X_train_observed.parquet'), pd.read_parquet('data/B/X_train_estimated.parquet').drop("date_calc", axis=1)]),
    pd.read_parquet('data/B/X_test_estimated.parquet').drop("date_calc", axis=1),
    "B"
)

Y_train_c, X_train_c, X_test_c, X_c, train_c = mf.full_clean(
    pd.read_parquet('data/C/train_targets.parquet'),
    pd.concat([pd.read_parquet('data/C/X_train_observed.parquet'), pd.read_parquet('data/C/X_train_estimated.parquet').drop("date_calc", axis=1)]),
    pd.read_parquet('data/C/X_test_estimated.parquet').drop("date_calc", axis=1),
    "C"
)

Y_train = pd.concat([Y_train_a, Y_train_b, Y_train_c], ignore_index=True)
X_train = pd.concat([X_train_a, X_train_b, X_train_c], ignore_index=True)
X_test = pd.concat([X_test_a, X_test_b, X_test_c], ignore_index=True)
X = pd.concat([X_a, X_b, X_c], ignore_index=True)
train = pd.concat([train_a, train_b, train_c], ignore_index=True)

In [10]:
j_Y_train_a = pd.read_parquet("Jørgen_cleaned_data/y_train_a.parquet")
j_Y_train_b = pd.read_parquet("Jørgen_cleaned_data/y_train_b.parquet")
j_Y_train_c = pd.read_parquet("Jørgen_cleaned_data/y_train_c.parquet")
j_Y_train = pd.concat([j_Y_train_a, j_Y_train_b, j_Y_train_c], ignore_index=True)

j_X_train_a = pd.read_parquet("Jørgen_cleaned_data/x_train_a.parquet")
j_X_train_b = pd.read_parquet("Jørgen_cleaned_data/x_train_b.parquet")
j_X_train_c = pd.read_parquet("Jørgen_cleaned_data/x_train_c.parquet")
j_X_train = pd.concat([j_X_train_a, j_X_train_b, j_X_train_c], ignore_index=True)

j_train_a = pd.concat([j_Y_train_a, j_X_train_a], axis = 1)
j_train_b = pd.concat([j_Y_train_b, j_X_train_b], axis = 1)
j_train_c = pd.concat([j_Y_train_c, j_X_train_c], axis = 1)

j_X_test_a = pd.read_parquet("Jørgen_cleaned_data/x_test_a.parquet")
j_X_test_b = pd.read_parquet("Jørgen_cleaned_data/x_test_b.parquet")
j_X_test_c = pd.read_parquet("Jørgen_cleaned_data/x_test_c.parquet")
j_X_test = pd.concat([j_X_test_a, j_X_test_b, j_X_test_c], ignore_index=True)

# Main stuff:

In [11]:
# fig_train, axis_train = plt.subplots(len(train_a.columns), figsize=(10, 5 * len(train_a.columns)))

# for i in range(len(train_a.columns)):
#     if i > 0:
#         axis_train[i].title.set_text(f"{i-1}: {train_a.columns[i]}")
#     else:
#         axis_train[i].title.set_text(f"{train_a.columns[i]}")
#     axis_train[i].plot(train_a.iloc[:,[i]], label = "A")
#     axis_train[i].plot(train_b.iloc[:,[i]], label = "B")
#     axis_train[i].plot(train_c.iloc[:,[i]], label = "C")
#     axis_train[i].legend(loc="upper left")
#     axis_train[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y/%m'))

# plt.tight_layout()
# fig_train.savefig('fig/train.pdf', pad_inches = .5)
# plt.close(fig_train)

# fig_test, axis_test = plt.subplots(len(X_test_a.columns), figsize=(10, 5 * len(X_test_a.columns)))

# for i in range(len(X_test_a.columns)):
#     axis_test[i].title.set_text(f"{i}: {X_test_a.columns[i]}")
#     axis_test[i].plot(X_test_a.iloc[:,[i]], label = "A")
#     axis_test[i].plot(X_test_a.iloc[:,[i]], label = "B")
#     axis_test[i].plot(X_test_a.iloc[:,[i]], label = "C")
#     axis_test[i].legend(loc="upper left")
#     axis_test[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y/%m'))

# plt.tight_layout()
# fig_test.savefig('fig/test.pdf', pad_inches = .5)
# plt.close(fig_test)

In [12]:
    # ignore_features = [
    #     "ceiling_height_agl__m",
    #     "cloud_base_agl__m",
    #     "snow_density__kgm3",
    # ]
    
    # categorical_features = [
    #     "dew_or_rime__idx",
    #     "elevation__m",
    #     "is_day__idx",
    #     "is_in_shadow__idx",
    #     "precip_type_5min__idx",
    #     # "snow_drift__idx", Almost no variation
    #     "wind_speed_w_1000hPa__ms"
    # ]
    
    # numeric_features = [
    #     "absolute_humidity_2m__gm3",
    #     "air_density_2m__kgm3",
    #     # "ceiling_height_agl__m", Fucked up
    #     "clear_sky_energy_1h__J",
    #     "clear_sky_rad__W",
    #     # "cloud_base_agl__m", Fucked up
    #     "dew_point_2m__K",
    #     "diffuse_rad__W",
    #     "diffuse_rad_1h__J",
    #     "direct_rad__W",
    #     "direct_rad_1h__J",
    #     "effective_cloud_cover__p",
    #     "fresh_snow_12h__cm",
    #     "fresh_snow_1h__cm",
    #     "fresh_snow_24h__cm",
    #     "fresh_snow_3h__cm",
    #     "fresh_snow_6h__cm",
    #     "msl_pressure__hPa",
    #     "precip_5min__mm",
    #     "pressure_100m__hPa",
    #     "pressure_50m__hPa",
    #     "prob_rime__p",
    #     "rain_water__kgm2",
    #     "relative_humidity_1000hPa__p",
    #     "sfc_pressure__hPa",
    #     # "snow_density__kgm3", Fucked up
    #     "snow_depth__cm",
    #     "snow_melt_10min__mm",
    #     "snow_water__kgm2",
    #     "sun_azimuth__d",
    #     "sun_elevation__d",
    #     "super_cooled_liquid_water__kgm2",
    #     "t_1000hPa__K",
    #     "total_cloud_cover__p",
    #     "visibility__m",
    #     "wind_speed_10m__ms",
    #     "wind_speed_u_10m__ms",
    #     "wind_speed_v_10m__ms",
    # ]
    
similar_features = {
    "clear_sky": ["clear_sky_energy_1h__J","clear_sky_rad__W"],
    "dew_point_2m__K_and_t_1000hPa__K": ["dew_point_2m__K", "t_1000hPa__K"],
    "diffuse_rad": ["diffuse_rad__W", "diffuse_rad_1h__J"],
    "direct_rad": ["direct_rad__W", "direct_rad_1h__J"],
    "fresh_snow__cm": ["fresh_snow_12h__cm", "fresh_snow_1h__cm", "fresh_snow_24h__cm", "fresh_snow_3h__cm", "fresh_snow_6h__cm"],
    "pressure__hPa": ["msl_pressure__hPa", "pressure_100m__hPa", "pressure_50m__hPa", "sfc_pressure__hPa"]
}

In [13]:
j_reg_full_a = RegressionExperiment()
j_reg_full_a.setup(
    data = j_train_a,
    use_gpu = True,
    target = "pv_measurement",
    train_size = .99999,
    max_encoding_ohe = -1,
    create_date_columns = [],
    low_variance_threshold = 0,
    # group_features = similar_features,
    # remove_multicollinearity = True,
    # drop_groups=True,
    fold = 2,
    # polynomial_features=True,
    # polynomial_degree = 3,
    # feature_selection=True,
    # n_features_to_select = 32,
    # pca = True,
    # html = False,
    session_id = 97,
)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are n

,Description,Value
0,Session id,97
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(18036, 164)"
4,Transformed data shape,"(18036, 160)"
5,Transformed train set shape,"(18035, 160)"
6,Transformed test set shape,"(1, 160)"
7,Numeric features,162
8,Date features,1
9,Rows with missing values,98.5%


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) Iris(R) Xe Graphics, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are n

In [42]:
j_lgbm_full_a = j_reg_full_a.create_model(
    estimator = "lightgbm",
    # fold = 30,
    verbose = False,
)
# j_lgbm_full_a_tuned = j_reg_full_a.tune_model(
#     estimator = j_lgbm_full_a,
#     # n_iter=20,
#     verbose = False,
# )
# j_lgbm_full_a_score = j_reg_full_a.pull().loc[["Mean", "Std"],["MAE"]].transpose()
# j_lgbm_full_a_score.columns.name = None
# j_lgbm_full_a_score.index = ["Lightgbm"]
# j_lgbm_full_a_score.index.name = "MAE"
# # display(j_lgbm_full_a_score)

j_cat_full_a = j_reg_full_a.create_model(
    estimator = "catboost",
    # fold = 30,
    verbose = False,
)
# j_cat_full_a_tuned = j_reg_full_a.tune_model(
#     estimator = j_cat_full_a,
#     # n_iter=20,
#     verbose = False,
# )
# j_cat_full_a_score = j_reg_full_a.pull().loc[["Mean", "Std"],["MAE"]].transpose()
# j_cat_full_a_score.columns.name = None
# j_cat_full_a_score.index = ["CatBoost"]
# j_cat_full_a_score.index.name = "MAE"
# # display(j_cat_full_a_score)

# j_blended_full_a = j_reg_full_a.blend_models(
#     estimator_list = [
#         j_lgbm_full_a,
#         j_cat_full_a,
#     ],
#     # fold = 30,
#     verbose = False,
# )
# j_blended_full_a_tuned = j_reg_full_a.tune_model(
#     estimator = j_blended_full_a,
#     # n_iter=20,
#     verbose = False,
# )
# j_blended_full_a_score = j_reg_full_a.pull().loc[["Mean", "Std"],["MAE"]].transpose()
# j_blended_full_a_score.columns.name = None
# j_blended_full_a_score.index = ["Blended"]
# j_blended_full_a_score.index.name = "MAE"
# # display(j_blended_full_a_score)

# j_full_a_score = pd.concat([j_lgbm_full_a_score, j_cat_full_a_score, j_blended_full_a_score])
# display(j_full_a_score.style.highlight_min("Mean", axis = 0))

j_et_full_a = j_reg_full_a.create_model(
    estimator = "et",
    # fold = 30,
    verbose = False,
)

In [ ]:
blender_soft = j_reg_full_a.blend_models(
    estimator_list = [
        j_lgbm_full_a,
        j_cat_full_a,
        #  et_full_a,
        #  rf_full_a,
    ],
    # fold = 10,
)
pull_a = j_reg_full_a.pull()

In [43]:
lgbm_full_feature_importance = pd.DataFrame(j_lgbm_full_a.feature_importances_, index=j_lgbm_full_a.feature_name_, columns = ["Importance"])
lgbm_full_feature_importance.sort_values(by="Importance", ascending=False, inplace=True)
lgbm_full_feature_importance_names = lgbm_full_feature_importance[round(lgbm_full_feature_importance["Importance"]) == 0].index.tolist()
len(lgbm_full_feature_importance_names)

31

In [44]:
cat_full_feature_importance = pd.DataFrame(j_cat_full_a.feature_importances_, index=j_cat_full_a.feature_names_, columns = ["Importance"])
cat_full_feature_importance.sort_values(by="Importance", ascending=False, inplace=True)
cat_full_feature_importance_names = cat_full_feature_importance[round(cat_full_feature_importance["Importance"]) == 0].index.tolist()
len(cat_full_feature_importance_names)

105

In [49]:
et_full_feature_importance = pd.DataFrame(j_et_full_a.feature_importances_, index=j_et_full_a.feature_names_in_, columns = ["Importance"])
et_full_feature_importance.sort_values(by="Importance", ascending=False, inplace=True)
et_full_feature_importance_names = et_full_feature_importance[round(et_full_feature_importance["Importance"]) == 0].index.tolist()
len(et_full_feature_importance_names)

159

In [64]:
len(list(set(["hei", "hello"]) & set(["hi"])))

0

In [66]:
print(["hei", "hello"].remove("hei"))

None


In [58]:
list(set(lgbm_full_feature_importance_names) & set(cat_full_feature_importance_names) & set(et_full_feature_importance_names))

['is_estimated',
 'fresh_snow_3hcm',
 'dew_or_rimeidx',
 'is_in_shadowidx_rolling_avg_3',
 'snow_melt_10minmm_rolling_avg_3',
 'fresh_snow_12hcm',
 'fresh_snow_24hcm_rolling_avg_3',
 'fresh_snow_1hcm',
 'prob_rimep_rolling_avg_5',
 'prob_rimep_rolling_avg_7',
 'snow_melt_10minmm_rolling_avg_5',
 'fresh_snow_3hcm_rolling_avg_3',
 'fresh_snow_12hcm_rolling_avg_5',
 'snow_depthcm_rolling_avg_3',
 'fresh_snow_3hcm_rolling_avg_7',
 'is_in_shadowidx',
 'fresh_snow_1hcm_rolling_avg_3',
 'snow_depthcm_rolling_avg_5',
 'snow_melt_10minmm',
 'fresh_snow_1hcm_rolling_avg_7',
 'is_dayidx',
 'fresh_snow_6hcm_rolling_avg_5',
 'fresh_snow_3hcm_rolling_avg_5',
 'prob_rimep_rolling_avg_3',
 'wind_speed_w_1000hPams',
 'snow_melt_10minmm_rolling_avg_7',
 'snow_depthcm_rolling_avg_7',
 'fresh_snow_6hcm',
 'fresh_snow_1hcm_rolling_avg_5',
 'prob_rimep',
 'fresh_snow_12hcm_rolling_avg_3']

In [ ]:
train_a_poly = reg_full_a.dataset_transformed
train_a_poly.columns = train_a_poly.columns.str.replace(" ", "_")
train_a_poly_red = train_a_poly[lgbm_full_feature_importance_names + ["pv_measurement"]]

In [ ]:
pipeline_full_a = reg_full_a.get_config('pipeline')
X_test_a_poly = pipeline_full_a.transform(X_test_a)
X_test_a_poly.columns = X_test_a_poly.columns.str.replace(" ", "_")
X_test_a_poly_red = X_test_a_poly[lgbm_full_feature_importance_names]

In [ ]:
reg_lgbm_a_full_feature_importance_names_a = RegressionExperiment()
reg_lgbm_a_full_feature_importance_names_a.setup(
    data = train_a_poly_red,
    # use_gpu = True,
    target = "pv_measurement",
    train_size = .99999,
    max_encoding_ohe = -1,
    create_date_columns = [],
    # low_variance_threshold = 0,
    # group_features = similar_features,
    # remove_multicollinearity = True,
    # drop_groups=True,
    fold = 20,
    session_id = 0,
    # polynomial_features=True,
    # polynomial_degree = 3,
    # feature_selection=True,
    # pca = True,
    html = False,
)

In [ ]:
lgbm_full_a_feature_importance_names_a = reg_lgbm_a_full_feature_importance_names_a.create_model("lightgbm")

In [ ]:
# plt.plot(reg_full_a.predict_model(lgbm_full_a, X_test_a)["prediction_label"])
plt.plot(reg_lgbm_a_full_feature_importance_names_a.predict_model(lgbm_full_a_feature_importance_names_a, X_test_a_poly_red)["prediction_label"])
plt.plot(pd.read_csv("pycaret_catboost_gbm_average.csv").iloc[:720]["prediction"])
plt.show()

In [ ]:
plt.plot(pd.read_csv("pycaret_advanced_rolling357.csv").iloc[:720]["prediction"])
plt.plot(pd.read_csv("pycaret_catboost_gbm_average.csv").iloc[:720]["prediction"])
# plt.plot(reg_full_a.predict_model(lgbm_full_a, X_test_a)["prediction_label"])

# plt.plot(pd.read_csv("pycaret_catboost.csv")["prediction"], "-")
# plt.plot(pd.read_csv("pycaret_advanced_rf.csv")["prediction"], "-")
plt.show()

In [ ]:
reg_full_b = RegressionExperiment()
reg_full_b.setup(
    data = train_b,
    # use_gpu = True,
    target = "pv_measurement",
    train_size = .99999,
    max_encoding_ohe = -1,
    create_date_columns = [],
    low_variance_threshold = 0,
    # group_features = similar_features,
    # remove_multicollinearity = True,
    # drop_groups=True,
    fold = 20,
    session_id = 0,
    polynomial_features=True,
    # polynomial_degree = 3,
    # feature_selection=True,
    # n_features_to_select = 20,
    # pca = True,
    # html = False,
)

In [ ]:
lgbm_full_b = reg_full_b.create_model("lightgbm")

In [ ]:
lgbm_full_b_feature_importance = pd.DataFrame(lgbm_full_b.feature_importances_, index=lgbm_full_b.feature_name_, columns = ["Importance"])
lgbm_full_b_feature_importance.sort_values(by="Importance", ascending=False, inplace=True)
lgbm_full_b_feature_importance_names = lgbm_full_b_feature_importance[lgbm_full_b_feature_importance["Importance"] > 3].index.tolist()
# lgbm_full_feature_importance_names

In [ ]:
train_b_poly = reg_full_b.dataset_transformed
train_b_poly.columns = train_b_poly.columns.str.replace(" ", "_")
train_b_poly_red = train_b_poly[lgbm_full_b_feature_importance_names + ["pv_measurement"]]

In [ ]:
train_b_poly_red

In [ ]:
reg_lgbm_full_b_feature_importance_names_a = RegressionExperiment()
reg_lgbm_full_b_feature_importance_names_a.setup(
    data = train_b_poly_red,
    # use_gpu = True,
    target = "pv_measurement",
    train_size = .99999,
    max_encoding_ohe = -1,
    create_date_columns = [],
    # low_variance_threshold = 0,
    # group_features = similar_features,
    # remove_multicollinearity = True,
    # drop_groups=True,
    fold = 20,
    session_id = 0,
    # polynomial_features=True,
    # polynomial_degree = 3,
    # feature_selection=True,
    # pca = True,
    # html = False,
)

In [ ]:
lgbm_full_feature_importance_names_a = reg_lgbm_full_feature_importance_names_a.create_model("lightgbm")

# shit

In [ ]:
reg_full = RegressionExperiment()
reg_full.setup(
    data = train,
    use_gpu = True,
    target = "pv_measurement",
    train_size = .99999,
    max_encoding_ohe = -1,
    create_date_columns = [],
    fold = 20,
    session_id = 0,
)

In [ ]:
cat_full = reg_full.create_model("catboost")

In [ ]:
ensembled_cat_full = reg_full.ensemble_model(
    cat_full,
    method = "Bagging",
    n_estimators = 20,
    # choose_better = True,
    optimize = "MSE"
)

In [ ]:
reg_full.evaluate_model(cat_full)

In [ ]:
cat_full_feature_importance = pd.DataFrame(cat_full.feature_importances_, index=cat_full.feature_names_, columns = ["Importance"])
cat_full_feature_importance.sort_values(by="Importance", ascending=False, inplace=True)
cat_full_feature_importance_names = cat_full_feature_importance.index.tolist()
# cat_full_feature_importance_names

In [ ]:
cat_full_feature_importance_names_selected = cat_full_feature_importance_names[:12]
reg_important_features = RegressionExperiment()
reg_important_features.setup(
    data = train,
    use_gpu = True,
    target = "pv_measurement",
    train_size = .99999,
    max_encoding_ohe = -1,
    create_date_columns = [],
    fold = 20,
    feature_selection = True,
    n_features_to_select = 1,
    keep_features = cat_full_feature_importance_names_selected,
    session_id = 0
)

In [ ]:
cat_important_features = reg_important_features.create_model("catboost")

In [ ]:
ensembled_cat_important_features = reg_important_features.ensemble_model(
    cat_important_features,
    method = "Bagging",
    n_estimators = 20,
    # choose_better = True,
    optimize = "MSE"
)

In [ ]:
final_cat_full = reg_full.finalize_model(ensembled_cat_full)
final_cat_important_features = reg_important_features.finalize_model(ensembled_cat_important_features)

In [ ]:
cat_full_predictions = reg_full.predict_model(final_cat_full, X_test)["prediction_label"]
# cat_full_predictions[X_test[~((X_test["time"].dt.hour <= 21) & (X_test["time"].dt.hour >= 1))].index] = 0
cat_full_predictions[cat_full_predictions < 0] = 0
# plt.plot(cat_full_predictions)
# plt.show()

In [ ]:
cat_important_features_predictions = reg_important_features.predict_model(final_cat_important_features, X_test)["prediction_label"]
# cat_important_features_predictions[X_test[~((X_test["time"].dt.hour <= 21) & (X_test["time"].dt.hour >= 1))].index] = 0
cat_important_features_predictions[cat_important_features_predictions < 0] = 0
# plt.plot(cat_important_features_predictions)
# plt.show()

In [ ]:
mf.export_csv2(
    np.array(cat_important_features_predictions),
    # f"PyCaret_CatBoost_7_20_Fold_20_Bag.csv"
    f"PyCaret_CatBoost_8_20_Fold_20_Bag_top_{len(cat_full_feature_importance_names_selected)}_features.csv"
)

In [ ]:
plt.plot(cat_full_predictions)
plt.plot(cat_important_features_predictions)
plt.show()

In [ ]:
# tuned_cat_full = reg_full.tune_model(
#     cat_full,
#     n_iter = 20,
#     # choose_better = False,
#     optimize = "MAE"
# )
# ensembled_tuned_cat_full = reg_full.ensemble_model(
#     tuned_cat_full,
#     method = "Bagging",
#     n_estimators = 20,
#     # choose_better = True,
#     optimize = "MAE"
# )

In [ ]:
lgbm_full = reg_full.create_model("lightgbm")

In [ ]:
ensembled_lgbm_full = reg_full.ensemble_model(
    lgbm_full,
    method = "Bagging",
    n_estimators = 20,
    # choose_better = True,
    optimize = "MAE"
)

In [ ]:
# tuned_lgbm_full = reg_full.tune_model(lgbm_full, n_iter = 20, optimize = "MSE")
# ensembled_tuned_lgbm_full = reg_full.ensemble_model(tuned_lgbm_full, method = "Bagging", n_estimators = 20, choose_better = True, optimize = "MSE")

In [ ]:
# reg_full.evaluate_model(cat_full)

In [ ]:
cat_full_predictions = reg_full.predict_model(cat_full, X_test)["prediction_label"]
# cat_full_predictions[X_test[~((X_test["time"].dt.hour <= 21) & (X_test["time"].dt.hour >= 1))].index] = 0
cat_full_predictions[cat_full_predictions < 0] = 0
plt.plot(cat_full_predictions)
plt.show()

In [ ]:
cat_full_feature_importance_names

In [ ]:
results_cat_important_features = np.array([])
for i in range(21):
   cat_full_feature_importance_selected_i = cat_full_feature_importance_names[:i+1]
   reg_important_features_i = RegressionExperiment()
   reg_important_features_i.setup(
      data = train,
      use_gpu = True,
      target = "pv_measurement",
      train_size = .99999,
      create_date_columns = [],
      fold = 10,
      feature_selection = True,
      n_features_to_select = 1,
      keep_features = cat_full_feature_importance_selected_i,
      session_id = 0,
      verbose = False)
   print(f"{cat_full_feature_importance_selected_i}")
   cat_important_features_i = reg_important_features_i.create_model("catboost")
   results_cat_important_features_i = reg_important_features_i.pull()
   MAE_i = results_cat_important_features_i[results_cat_important_features_i.index == "Mean"]["MAE"]
   results_cat_important_features = np.append(results_cat_important_features, MAE_i)

In [ ]:
plt.plot(results_cat_important_features)
plt.show()

In [ ]:
reg_important_features = RegressionExperiment()
reg_important_features.setup(data = train,
                     use_gpu = True,
                     target = "pv_measurement",
                     train_size = .99999,
                     create_date_columns = [],
                    #  create_date_columns = ["day", "month"],
                    #  polynomial_features = True,
                     fold = 10,
                     feature_selection = True,
                     n_features_to_select = 1,
                     keep_features = cat_full_feature_importance_selected[:10],
                     session_id = 0)

In [ ]:
cat_important_features = reg_important_features.create_model("catboost")

In [ ]:
tuned_cat_important_features = reg_important_features.tune_model(cat_important_features, n_iter = 50, optimize = "MAE")
bagged_cat_important_features = reg_important_features.ensemble_model(tuned_cat_important_features, method = "Bagging")

In [ ]:
cat_important_features_predictions = reg_full.predict_model(cat_important_features, X_test)["prediction_label"]
cat_important_features_predictions[X_test[~((X_test["time"].dt.hour <= 21) & (X_test["time"].dt.hour >= 1))].index] = 0
cat_important_features_predictions[cat_important_features_predictions < 0] = 0
plt.plot(cat_important_features_predictions)
plt.show()

In [ ]:
reg_full.predict_model(bagged_cat_important_features, X_test)["prediction_label"]

# A:

In [ ]:
reg_full_a = RegressionExperiment()
reg_full_a.setup(data = train_a,
               use_gpu = True,
               target = "pv_measurement",
               train_size = .99999,
               create_date_columns = [],
               # polynomial_features = True,
               fold = 10,
               ignore_features = ["location"],
               session_id = 0)

In [ ]:
cat_full_a = reg_full_a.create_model("catboost")

In [ ]:
reg_full_a.evaluate_model(cat_full_a)

In [ ]:
cat_full_predictions_a = reg_full_a.predict_model(cat_full_a, X_test_a)["prediction_label"]
cat_full_predictions_a[X_test_a[~((X_test_a["time"].dt.hour <= 21) & (X_test_a["time"].dt.hour >= 1))].index] = 0
cat_full_predictions_a[cat_full_predictions_a < 0] = 0
plt.plot(cat_full_predictions_a)
plt.show()

In [ ]:
cat_full_feature_importance_a = pd.DataFrame(cat_full_a.feature_importances_, index=cat_full_a.feature_names_, columns = ["Importance"])
cat_full_feature_importance_a.sort_values(by="Importance", ascending=False, inplace=True)
cat_full_feature_importance_selected_a = cat_full_feature_importance_a.iloc[:15].index.tolist()
cat_full_feature_importance_selected_a

In [ ]:
reg_important_features_a = RegressionExperiment()
reg_important_features_a.setup(data = train_a,
                     use_gpu = True,
                     target = "pv_measurement",
                     train_size = .99999,
                     create_date_columns = [],
                    #  create_date_columns = ["day", "month"],
                    #  polynomial_features = True,
                     fold = 10,
                     feature_selection = True,
                     n_features_to_select = 1,
                     keep_features = cat_full_feature_importance_selected_a,
                     session_id = 0)

In [ ]:
cat_important_features_a = reg_important_features_a.create_model("catboost")

In [ ]:
cat_full_predictions_a = reg_full_a.predict_model(cat_full_a, X_test_a)["prediction_label"]
cat_full_predictions_a[X_test_a[~((X_test_a["time"].dt.hour <= 21) & (X_test_a["time"].dt.hour >= 1))].index] = 0
cat_full_predictions_a[cat_full_predictions_a < 0] = 0
plt.plot(cat_full_predictions_a)
plt.show()

In [ ]:
sns.scatterplot(x = (X_train_a["sun_elevation__d"] * X_train_a["sun_azimuth__d"]), y = Y_train_a["pv_measurement"])
plt.show()
sns.scatterplot(x = np.tan(np.log(X_train_a["sun_elevation__d"] * X_train_a["sun_azimuth__d"])), y = Y_train_a["pv_measurement"])
plt.show()
sns.scatterplot(x = (X_train_a["sun_elevation__d"]), y = Y_train_a["pv_measurement"])
plt.show()

# B:

In [ ]:
reg_full_b = RegressionExperiment()
reg_full_b.setup(data = train_b,
               use_gpu = True,
               target = "pv_measurement",
               train_size = .99999,
               create_date_columns = [],
               # polynomial_features = True,
               fold = 10,
               ignore_features = ["location"],
               session_id = 0)

In [ ]:
cat_full_b = reg_full_b.create_model("catboost")

In [ ]:
reg_full_b.evaluate_model(cat_full_b)

In [ ]:
cat_full_predictions_b = reg_full_b.predict_model(cat_full_b, X_test_b)["prediction_label"]
cat_full_predictions_b[X_test_b[~((X_test_b["time"].dt.hour <= 21) & (X_test_b["time"].dt.hour >= 1))].index] = 0
cat_full_predictions_b[cat_full_predictions_b < 0] = 0
plt.plot(cat_full_predictions_b)
plt.show()

# C:

In [ ]:
reg_full_c = RegressionExperiment()
reg_full_c.setup(data = train_c,
               use_gpu = True,
               target = "pv_measurement",
               train_size = .99999,
               create_date_columns = [],
            #    polynomial_features = True,
               fold = 10,
               ignore_features = ["location"],
               session_id = 0)

In [ ]:
cat_full_c = reg_full_c.create_model("catboost")

In [ ]:
reg_full_c.evaluate_model(cat_full_c)

In [ ]:
cat_full_predictions_c = reg_full_c.predict_model(cat_full_c, X_test_c)["prediction_label"]
cat_full_predictions_c[X_test_c[~((X_test_c["time"].dt.hour <= 21) & (X_test_c["time"].dt.hour >= 1))].index] = 0
cat_full_predictions_c[cat_full_predictions_c < 0] = 0
plt.plot(cat_full_predictions_c)
plt.show()

# Export

In [ ]:
mf.export_csv2(np.array(cat_important_features_predictions), f"PyCaret_CatBoost_6_10_Fold_top_{len(cat_full_feature_importance_selected)}_features_hour_week_month_Make_zero_at_night.csv")